# Day 19
## Part 1

In [8]:
import numpy as np

In [26]:
scanners = []

with open("ex1.txt") as f:
    for scanner in f.read().split("\n\n"):
        field = []
        coords = scanner.split(" ---\n")[1]
        # print(coords)
        for line in coords.split("\n"):
            line = line.strip()
            coord_list = np.array([int(n) for n in line.split(",")])
            field.append(coord_list)
        scanners.append(np.array(field))
        
print(scanners[0][0:5])
print(scanners[0][:, 0][0:5])
print(np.stack([scanners[0][:, 0][0:5], scanners[0][:, 1][0:5], scanners[0][:, 2][0:5]], axis=1))

[[ 404 -588 -901]
 [ 528 -643  409]
 [-838  591  734]
 [ 390 -675 -793]
 [-537 -823 -458]]
[ 404  528 -838  390 -537]
[[ 404 -588 -901]
 [ 528 -643  409]
 [-838  591  734]
 [ 390 -675 -793]
 [-537 -823 -458]]


In [41]:
# I have a list of multiple numpy arrays
# Each numpy array is an array of coordinates
# I need to be able to get a list of rotations of this

def gen_rotations(coord_array):
    # arrays of all x, y, z coords
    x = coord_array[:, 0]
    y = coord_array[:, 1]
    z = coord_array[:, 2]
    
    rots = [np.stack([x, y, z], axis=1), np.stack([x, -y, -z], axis=1), 
            np.stack([-x, y, -z], axis=1), np.stack([-x, -y, z], axis=1), 
            np.stack([x, z, -y], axis=1), np.stack([x, -z, y], axis=1),
            np.stack([-x, z, y], axis=1), np.stack([-x, -z, -y], axis=1),
            np.stack([y, z, x], axis=1), np.stack([y, -z, -x], axis=1),
            np.stack([-y, z, -x], axis=1), np.stack([-y, -z, x], axis=1),
            np.stack([y, x, -z], axis=1), np.stack([y, -x, z], axis=1),
            np.stack([-y, x, z], axis=1), np.stack([-y, -x, -z], axis=1),
            np.stack([z, x, y], axis=1), np.stack([z, -x, -y], axis=1), 
            np.stack([-z, x, -y], axis=1), np.stack([-z, -x, y], axis=1), 
            np.stack([z, y, -x], axis=1), np.stack([z, -y, x], axis=1), 
            np.stack([-z, y, x], axis=1), np.stack([-z, -y, -x], axis=1)]
    
    return rots

test = scanners[0][0:5]
print(test)
print("rotations:")

rot_set = set()
rotations = gen_rotations(test)
for rot in rotations:
    rot_set.add(tuple(rot[0]))
    
print(len(rot_set))

[[ 404 -588 -901]
 [ 528 -643  409]
 [-838  591  734]
 [ 390 -675 -793]
 [-537 -823 -458]]
rotations:
24


Ok, I can now rotate each list of points so that I get 24 different ways corresponding to the 24 ways the scanner could be facing and rotated.

I now need a way to go and centre an array according to each point in it i.e. in turn, set each point as the origin.

In [46]:
def gen_origins(coord_array):
    for xyz in coord_array:  # for each [x, y, z]
        yield coord_array - xyz  # subtract that from each of them
        
for i in gen_origins(test):
    print(i)

[[    0     0     0]
 [  124   -55  1310]
 [-1242  1179  1635]
 [  -14   -87   108]
 [ -941  -235   443]]
[[ -124    55 -1310]
 [    0     0     0]
 [-1366  1234   325]
 [ -138   -32 -1202]
 [-1065  -180  -867]]
[[ 1242 -1179 -1635]
 [ 1366 -1234  -325]
 [    0     0     0]
 [ 1228 -1266 -1527]
 [  301 -1414 -1192]]
[[   14    87  -108]
 [  138    32  1202]
 [-1228  1266  1527]
 [    0     0     0]
 [ -927  -148   335]]
[[ 941  235 -443]
 [1065  180  867]
 [-301 1414 1192]
 [ 927  148 -335]
 [   0    0    0]]


Now I need a way to basically compare two arrays and see if they have at least 12 rows that are the same. If so, then they have 12 overlapping points.

In [283]:
test = np.array([[1, 2], [3, 4], [5, 6]])
test2 = np.array([[1, 2], [3, 4], [7, 8]])
# print(np.all(np.isin(test, test2), axis=1))

# def count_same_rows(array1, array2):
#     # THIS DOESN'T WORK RIGHT
#     # JUST CHECKS FOR ELEMENTS INDIVIDUALLY INSTEAD OF ROWS
#     # HMMMM
#     return np.sum(np.all(np.isin(array1, array2), axis=1))

def get_shared_rows(array1, array2):
    return set(tuple(i) for i in array1) & set(tuple(i) for i in array2)

def count_same_rows(array1, array2):
    # Creates a set of the rows of array1, then rows of array2, then intersection of them both
    shared = get_shared_rows(array1, array2)
    return len(shared)

print(count_same_rows(test, test2))

2


The problem says that for the examples, scanner 0 and scanner 1 have 12 points overlapping. Let's try to find them.

Maybe I should change my logic.

How about I create a dictionary tracking the coordinates of every scanner that I know? Then, I only work comparing the rest of the scanners to a scanner that I know the coordinates of. That way, I can use the fact I know its coordinates to help :O

The only problem then is that I need to work out how to find the coordinate of a scanner from a pair of coordinates that I know are equivalent, but I can at least get those coordinates during the loop usingenumerate above, I think. Maybe.
Will come back to this.

In [284]:
def match_fields(field1, field2, match_min=12, testing=False):
    """
    Match field2 to field1 by rotating and trying to match each probe.
    Returns (field2 rotated to the direction that matches, origin of field2 with respect to the origin of field1)
    """
    for field2_rot in gen_rotations(field2):
        # rotate field2 to some orientation
        for index_f1, field1_centred in enumerate(gen_origins(field1)):
            # centre field1 wrt each of its probes
            for index_f2, field2_rot_centred in enumerate(gen_origins(field2_rot)):
                # centre field2 wrt each of its probes
                # at some point, the probe that they are centred wrt will be the same one if they overlap
                if count_same_rows(field1_centred, field2_rot_centred) >= match_min:
                    # If over the limit matches are found, then they overlap
                    # Get the coords of this matching probe in each coordinate system
                    
                    # print("this is the function talking!")
                    # print("field1_centred and field2_rot_centred: (there should be at least 12 matches)")
                    # print(field1_centred)
                    # print(field2_rot_centred)
                    # print("done talking")
                    # print()
                    
                    orig_probe1 = field1[index_f1]
                    orig_probe2 = field2_rot[index_f2]
                    
                    origin_field2 = orig_probe1 - orig_probe2  # origin of field2 wrt field1
                    
                    # now, return the correctly rotated field2 and its origin
                    if testing:
                        return (field2_rot, origin_field2, field1_centred, field2_rot_centred,
                               count_same_rows(field1_centred, field2_rot_centred))
                    else:
                        return (field2_rot, origin_field2)
    # if here is reached, then no matches are found
    return None

match = match_fields(scanners[0], scanners[1])
print(match[0])
print()
print(correct_fields["scanner1"])
print(match[1])
# this correctly gets the centre! nice

[[-686  422 -578]
 [-605  423 -415]
 [-515  917  361]
 [ 336  658 -858]
 [ -95  138  -22]
 [ 476  619 -847]
 [ 340 -569  846]
 [-567 -361 -727]
 [ 460  603  452]
 [-669 -402 -600]
 [-729  430 -532]
 [ 500 -761 -534]
 [ 322  571 -750]
 [ 466 -666  811]
 [ 429 -592 -574]
 [ 355  545  477]
 [-703 -491  529]
 [ 328 -685 -520]
 [-413  935  424]
 [ 391  539  444]
 [-586 -435 -557]
 [ 364 -763  893]
 [-807 -499  711]
 [-755 -354  619]
 [-553  889  390]]

[[-686  422 -578]
 [-605  423 -415]
 [-515  917  361]
 [ 336  658 -858]
 [ -95  138  -22]
 [ 476  619 -847]
 [ 340 -569  846]
 [-567 -361 -727]
 [ 460  603  452]
 [-669 -402 -600]
 [-729  430 -532]
 [ 500 -761 -534]
 [ 322  571 -750]
 [ 466 -666  811]
 [ 429 -592 -574]
 [ 355  545  477]
 [-703 -491  529]
 [ 328 -685 -520]
 [-413  935  424]
 [ 391  539  444]
 [-586 -435 -557]
 [ 364 -763  893]
 [-807 -499  711]
 [-755 -354  619]
 [-553  889  390]]
[   68 -1246   -43]


In [285]:
known_centres = {"scanner0": np.array([0, 0, 0])}
correct_fields = {"scanner0": scanners[0]}
fields = {f"scanner{index}": field for index, field in enumerate(scanners)}
compared = set()

while len(known_centres.keys()) < len(fields.keys()):  # while not all fields are known
    print("New run. known_centres:")
    print([i for i in known_centres.keys()])
    old_correct_fields = {key: value for key, value in correct_fields.items()}
    for field_name, field in fields.items():
        if field_name in old_correct_fields.keys():
            continue  # already found a correct_field
        # compare this field to each field I already know the correct rotation of
        for correct_name, correct in old_correct_fields.items():
            if field_name + correct_name in compared:
                # already been compared
                continue
            match = match_fields(correct, field)
            if match is None:
                # no match found
                compared.add(field_name + correct_name)
                continue
            else:
                # match has been found
                compared.add(field_name + correct_name)
                rotated_field, new_origin = match
                correct_fields[field_name] = rotated_field
                print("What are the origins though?? :/")
                print(f"I think {correct_name} has origin {known_centres[correct_name]}")
                print(f"I think {field_name} has origin {new_origin} wrt above.")
                print(f"So wrt scanner0, has origin {new_origin + known_centres[correct_name]}?")
                print()
                known_centres[field_name] = new_origin + known_centres[correct_name]

New run. known_centres:
['scanner0']
What are the origins though?? :/
I think scanner0 has origin [0 0 0]
I think scanner1 has origin [   68 -1246   -43] wrt above.
So wrt scanner0, has origin [   68 -1246   -43]?

New run. known_centres:
['scanner0', 'scanner1']
What are the origins though?? :/
I think scanner1 has origin [   68 -1246   -43]
I think scanner3 has origin [ -160 -1134    23] wrt above.
So wrt scanner0, has origin [  -92 -2380   -20]?

What are the origins though?? :/
I think scanner1 has origin [   68 -1246   -43]
I think scanner4 has origin [ -88  113 1104] wrt above.
So wrt scanner0, has origin [  -20 -1133  1061]?

New run. known_centres:
['scanner0', 'scanner1', 'scanner3', 'scanner4']
What are the origins though?? :/
I think scanner4 has origin [  -20 -1133  1061]
I think scanner2 has origin [1125  -72  168] wrt above.
So wrt scanner0, has origin [ 1105 -1205  1229]?



In [286]:
for k, v in known_centres.items():
    print(k, v)
# these are correct! poggers

scanner0 [0 0 0]
scanner1 [   68 -1246   -43]
scanner3 [  -92 -2380   -20]
scanner4 [  -20 -1133  1061]
scanner2 [ 1105 -1205  1229]


Now I have the centres, I can use the correct_fields thing to just get a set of all probes and then cound the length of that set.

In [297]:
probes = set()
for name, field in correct_fields.items():
    moved_field = field + known_centres[name]
    for row in moved_field:
        probes.add(tuple(row))
        
print(len(probes))

79


Now I'm getting the right answer for the example, time to do the same with the actual input.

In [338]:
scanners = []

with open("scanners.txt") as f:
    for scanner in f.read().split("\n\n"):
        field = []
        coords = scanner.split(" ---\n")[1]
        # print(coords)
        for line in coords.split("\n"):
            line = line.strip()
            coord_list = np.array([int(n) for n in line.split(",")])
            field.append(coord_list)
        scanners.append(np.array(field))
        
print(scanners[0][0:5])
print(scanners[0][:, 0][0:5])
print(np.stack([scanners[0][:, 0][0:5], scanners[0][:, 1][0:5], scanners[0][:, 2][0:5]], axis=1))

[[ 497 -409 -534]
 [-367  419 -378]
 [  42  122   52]
 [-639 -583 -417]
 [-698 -415  664]]
[ 497 -367   42 -639 -698]
[[ 497 -409 -534]
 [-367  419 -378]
 [  42  122   52]
 [-639 -583 -417]
 [-698 -415  664]]


In [339]:
known_centres = {"scanner0": np.array([0, 0, 0])}
correct_fields = {"scanner0": scanners[0]}
fields = {f"scanner{index}": field for index, field in enumerate(scanners)}
compared = set()

while len(known_centres.keys()) < len(fields.keys()):  # while not all fields are known
    print("New run. known_centres:")
    print([i for i in known_centres.keys()])
    old_correct_fields = {key: value for key, value in correct_fields.items()}
    for field_name, field in fields.items():
        if field_name in correct_fields.keys():
            continue  # already found a correct_field
        # compare this field to each field I already know the correct rotation of
        for correct_name, correct in old_correct_fields.items():
            if field_name + correct_name in compared:
                # already been compared
                continue
            match = match_fields(correct, field)
            if match is None:
                # no match found
                compared.add(field_name + correct_name)
                continue
            else:
                # match has been found
                compared.add(field_name + correct_name)
                rotated_field, new_origin = match
                correct_fields[field_name] = rotated_field
                print(f"I think {correct_name} has origin {known_centres[correct_name]} and I think {field_name} has origin {new_origin} wrt above.")
                print(f"So wrt scanner0, {field_name} has origin {new_origin + known_centres[correct_name]}?")
                known_centres[field_name] = new_origin + known_centres[correct_name]

New run. known_centres:
['scanner0']
I think scanner0 has origin [0 0 0] and I think scanner7 has origin [1149  103  -42] wrt above.
So wrt scanner0, scanner7 has origin [1149  103  -42]?
I think scanner0 has origin [0 0 0] and I think scanner25 has origin [ -27  -23 1101] wrt above.
So wrt scanner0, scanner25 has origin [ -27  -23 1101]?
New run. known_centres:
['scanner0', 'scanner7', 'scanner25']
I think scanner25 has origin [ -27  -23 1101] and I think scanner3 has origin [ -81 1273  -33] wrt above.
So wrt scanner0, scanner3 has origin [-108 1250 1068]?
I think scanner7 has origin [1149  103  -42] and I think scanner34 has origin [1108  -34   52] wrt above.
So wrt scanner0, scanner34 has origin [2257   69   10]?
I think scanner7 has origin [1149  103  -42] and I think scanner35 has origin [   88 -1241   -64] wrt above.
So wrt scanner0, scanner35 has origin [ 1237 -1138  -106]?
New run. known_centres:
['scanner0', 'scanner7', 'scanner25', 'scanner3', 'scanner34', 'scanner35']
I thin

In [340]:
probes = set()
for name, field in correct_fields.items():
    moved_field = field + known_centres[name]
    for row in moved_field:
        probes.add(tuple(row))
        
print(len(probes))

436


In [341]:
with open("scanner_centres.txt", "w") as f:
    for value in known_centres.values():
        f.write(f"{value}\n")

## Part 2

In [342]:
import itertools

manhattans = []
print(known_centres.values())
for scanner1, scanner2 in itertools.combinations(known_centres.values(), 2):
    dx = abs(scanner1[0] - scanner2[0])
    dy = abs(scanner1[1] - scanner2[1])
    dz = abs(scanner1[2] - scanner2[2])
    manhattans.append(dx + dy + dz)

print(max(manhattans))

dict_values([array([0, 0, 0]), array([1149,  103,  -42]), array([ -27,  -23, 1101]), array([-108, 1250, 1068]), array([2257,   69,   10]), array([ 1237, -1138,  -106]), array([-1271,  1311,  1126]), array([ 2394, -1151,   -62]), array([1133, 1273, 1155]), array([ 1191, -2268,    31]), array([ 2355,   107, -1170]), array([3533,   84,  -61]), array([2365,   65, 1259]), array([ 3551, -1103,  -120]), array([4824,   77,  -25]), array([2370, 1147, 1153]), array([ 2292, -1138, -1190]), array([1169, 2485, 1247]), array([2289,   26, 2313]), array([ 3553,   109, -1140]), array([ 4698, -1107,    16]), array([4712,   94, 1119]), array([ 3581,  1204, -1150]), array([2277, 2508, 1196]), array([4754, 1284,  -97]), array([ 5936, -1213,   -61]), array([5955,  130, 1264]), array([3483, 2515, 1129]), array([ 4665, -1202,  1183]), array([4767, 2501, -133]), array([3604, 2493, 2325]), array([4825, 3706,  -67]), array([7102,   95, 1101]), array([4734, 2411, 1178]), array([5856, 2500, 1165]), array([4649, 36

Try doing part 2 for the example.

In [333]:
scanners = []

with open("ex1.txt") as f:
    for scanner in f.read().split("\n\n"):
        field = []
        coords = scanner.split(" ---\n")[1]
        # print(coords)
        for line in coords.split("\n"):
            line = line.strip()
            coord_list = np.array([int(n) for n in line.split(",")])
            field.append(coord_list)
        scanners.append(np.array(field))
        
print(scanners[0][0:5])
print(scanners[0][:, 0][0:5])
print(np.stack([scanners[0][:, 0][0:5], scanners[0][:, 1][0:5], scanners[0][:, 2][0:5]], axis=1))

[[ 404 -588 -901]
 [ 528 -643  409]
 [-838  591  734]
 [ 390 -675 -793]
 [-537 -823 -458]]
[ 404  528 -838  390 -537]
[[ 404 -588 -901]
 [ 528 -643  409]
 [-838  591  734]
 [ 390 -675 -793]
 [-537 -823 -458]]


In [334]:
known_centres = {"scanner0": np.array([0, 0, 0])}
correct_fields = {"scanner0": scanners[0]}
fields = {f"scanner{index}": field for index, field in enumerate(scanners)}
compared = set()

while len(known_centres.keys()) < len(fields.keys()):  # while not all fields are known
    print("New run. known_centres:")
    print([i for i in known_centres.keys()])
    old_correct_fields = {key: value for key, value in correct_fields.items()}
    for field_name, field in fields.items():
        if field_name in correct_fields.keys():
            continue  # already found a correct_field
        # compare this field to each field I already know the correct rotation of
        for correct_name, correct in old_correct_fields.items():
            if field_name + correct_name in compared:
                # already been compared
                continue
            match = match_fields(correct, field)
            if match is None:
                # no match found
                compared.add(field_name + correct_name)
                continue
            else:
                # match has been found
                compared.add(field_name + correct_name)
                rotated_field, new_origin = match
                correct_fields[field_name] = rotated_field
                # print(f"I think {correct_name} has origin {known_centres[correct_name]} and I think {field_name} has origin {new_origin} wrt above.")
                # print(f"So wrt scanner0, has origin {new_origin + known_centres[correct_name]}?")
                # print()
                known_centres[field_name] = new_origin + known_centres[correct_name]

New run. known_centres:
['scanner0']
New run. known_centres:
['scanner0', 'scanner1']
New run. known_centres:
['scanner0', 'scanner1', 'scanner3', 'scanner4']


In [335]:
probes = set()
for name, field in correct_fields.items():
    moved_field = field + known_centres[name]
    for row in moved_field:
        probes.add(tuple(row))
        
print(len(probes))

79


In [336]:
manhattans = []
print(known_centres.values())
for scanner1, scanner2 in itertools.combinations(known_centres.values(), 2):
    dx = abs(scanner1[0] - scanner2[0])
    dy = abs(scanner1[1] - scanner2[1])
    dz = abs(scanner1[2] - scanner2[2])
    manhattans.append(dx + dy + dz)

print(max(manhattans))

dict_values([array([0, 0, 0]), array([   68, -1246,   -43]), array([  -92, -2380,   -20]), array([  -20, -1133,  1061]), array([ 1105, -1205,  1229])])
3621
